In [86]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [87]:
# 파일 로드
df_welfare = pd.read_csv('./welfare_school_concat.csv')
df_safety_sign = pd.read_csv("./서울시_안전표지_관련정보.csv",encoding='euc-kr')

C:\Users\yeeun\AppData\Local\Temp\ipykernel_10192\4173310745.py:3: DtypeWarning: Columns (0,1,10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_safety_sign = pd.read_csv("./서울시_안전표지_관련정보.csv",encoding='euc-kr')


In [88]:
# 좌표 데이터를 Shapely Point 객체로 변환
gdf_welfare = gpd.GeoDataFrame(df_welfare, geometry=[Point(xy) for xy in zip(df_welfare['X'], df_welfare['Y'])])
gdf_safety_sign = gpd.GeoDataFrame(df_safety_sign, geometry=[Point(xy) for xy in zip(df_safety_sign['X좌표'], df_safety_sign['Y좌표'])])

In [89]:
# GeoDataFrame에 좌표계 설정
gdf_welfare.set_crs(epsg=5186, inplace=True)
gdf_safety_sign.set_crs(epsg=5186, inplace=True)

설치방식 상태 (공통) 표지코드    비고           X좌표           Y좌표       설치일  \
0       003     001  002        197430.82666  551261.04855  20130503   
1       999     001  003  T형설치  198212.57833  553261.42560             
2       003     001             200019.71305  554940.85846             
3       003     001  003        199996.21210  554853.25479  20141109   
4       003     001  002        199994.21744  554963.77265  20141109   
...     ...     ...  ...   ...           ...           ...       ...   
349995  003          002        200206.09163  549129.25680             
349996  003          002        199043.00000  551338.78125             
349997  002          002        198998.59314  551529.46674             
349998  001     001  002        198959.17804  551573.54977             
349999  001     001  002        198959.73857  551572.89364             

             교체일         표지관리번호         지주관리번호  ... 작업구분 (공통)  표출구분 (공통)  \
0       20130503  04-0000292950  02-0000145469  ...         1          2   
1                 04-0000294826  02-0000178517  ...         6          2   
2                 04-0000296447  02-0000149786  ...         1          1   
3       20141109  04-0000129832  02-0000074717  ...         1          2   
4       20141109  04-0000296302  02-0000149781  ...         6          2   
...          ...            ...            ...  ...       ...        ...   
349995            04-0000325750  02-0000167228  ...         1          2   
349996            04-0000325751  02-0000190838  ...         1          2   
349997            04-0000325752  02-0000013021  ...         1          2   
349998            04-0000325753  02-0000124864  ...         1          2   
349999            04-0000325754  02-0000124864  ...         1          2   

        신규정규화ID    이력ID         공사관리번호   표지관리번호.1 표지인덱스 위치정보 공사형태  \
0       3259401  313963  2013-0208-003  04-292950   322       001   
1       3373001  315839  2013-0208-001  04-294826   428       001   
2       4306633  317460                 04-296447   324       001   
3                 99842  2014-0208-031  04-129832   751    9  001   
4       4306641  317315  2014-0208-031  04-296302   219       001   
...         ...     ...            ...        ...   ...  ...  ...   
349995           346763                 04-325750   111       001   
349996           346764                 04-325751   750       001   
349997           346765                 04-325752   315       001   
349998           346766                 04-325753   132       001   
349999           346767                 04-325754   226       001   

                             geometry  
0       POINT (197430.827 551261.049)  
1       POINT (198212.578 553261.426)  
2       POINT (200019.713 554940.858)  
3       POINT (199996.212 554853.255)  
4       POINT (199994.217 554963.773)  
...                               ...  
349995  POINT (200206.092 549129.257)  
349996  POINT (199043.000 551338.781)  
349997  POINT (198998.593 551529.467)  
349998  POINT (198959.178 551573.550)  
349999  POINT (198959.739 551572.894)  

[350000 rows x 21 columns]

In [90]:
# 복지시설의 각 위치에 대해 300미터 버퍼 생성
gdf_welfare['buffer'] = gdf_welfare.geometry.buffer(300)

In [91]:
# traffic equipment 개수를 계산하는 함수
def count_facilities_within_buffer(row, gdf_b):
    # 버퍼 내의 traffic equipment 필터링
    facilities_within_buffer = gdf_b[gdf_b.geometry.within(row.buffer)]
    return len(facilities_within_buffer)

In [92]:
# '안전표지 개수' 열 추가
gdf_welfare['안전표지 개수'] = gdf_welfare.apply(lambda row: count_facilities_within_buffer(row, gdf_safety_sign), axis=1)

In [93]:
safety_sign_count = gdf_welfare[['시설명', '안전표지 개수']].copy()
print(safety_sign_count)


              시설명  안전표지 개수
0        송파구방이복지관      294
1      서울시각장애인복지관      318
2    서울시립발달장애인복지관       10
3           청음복지관      278
4      마포장애인종합복지관      196
..            ...      ...
704        서울정문학교      224
705        서울정진학교       55
706        한국우진학교      165
707         서울농학교      273
708         서울맹학교      217

[709 rows x 2 columns]
